In [1]:
""" this notebook contains etl for insidelacrosse club ranking data """

' this notebook contains etl for insidelacrosse club ranking data '

In [2]:
import pandas as pd
import os
import sys
sys.path.insert(3,'../etl/')
from data_helper import get_csv_dataframes_from_folder
from data_helper import set_col_names_lower,set_col_names_underscore_separated
from data_helper import drop_unnamed_columns

from urllib.parse import quote
from sqlalchemy.engine import create_engine

In [33]:
WRITE_LOCAL_FLAG=True
WRITE_DB_FLAG=True

### Extract

In [30]:
# extrat data from raw folder
raw_folder_path = '../data/extra/clubranking/insidelacrosse/output/raw/'
bronze_folder_path = '../data/extra/clubranking/insidelacrosse/output/bronze/'
bronze_file_name = 'clubsranking.csv'
dataframe_dict=get_csv_dataframes_from_folder(raw_folder_path)

#### Transform

In [35]:
dfs_list = []
for file_name, dataframe in dataframe_dict.items():
    clas,season,gender=file_name.split('.csv')[0].split('_')
    dataframe['class']=clas
    dataframe['season']=season
    dataframe['gender']=gender
    dataframe['websource']='insidelacrosse'
    dataframe=drop_unnamed_columns(set_col_names_underscore_separated(set_col_names_lower(dataframe)))
    dfs_list.append(dataframe)
    
clubs_ranking_df=pd.concat(dfs_list)
clubs_ranking_df

,rank,club,rating,link,class,season,gender,websource
0,--,214LAX DALLAS 2026 VOLT,--,NaN,2026,2024,girls,insidelacrosse
1,--,315 Elite 2026 Green,--,NaN,2026,2024,girls,insidelacrosse
2,--,Common Goal 2026,--,NaN,2026,2024,girls,insidelacrosse
3,--,3d New England 2026,--,NaN,2026,2024,girls,insidelacrosse
4,--,3d Virginia Metro 2026 Navy,--,NaN,2026,2024,girls,insidelacrosse
...,...,...,...,...,...,...,...,...
19,--,NH Tomahawks 2024 Purple,--,NaN,2024,2020,girls,insidelacrosse
0,--,2 Knights 2028,--,NaN,2028,2019,boys,insidelacrosse
1,--,CT Chargers - 2028,--,NaN,2028,2019,boys,insidelacrosse
0,--,Sweetlax FL 2025,--,NaN,2025,2020,boys,insidelacrosse


### Load

In [36]:
if WRITE_LOCAL_FLAG:
    clubs_ranking_df.to_csv(os.path.join(bronze_folder_path,bronze_file_name),index=False)
    print(f'{bronze_file_name} saved sucess into bronze folder')


clubsranking.csv saved sucess into bronze folder


In [37]:
if WRITE_DB_FLAG:
    host = '182.75.105.186'
    dev_port = '33096'
    dev_database = 'lacrosse-pre-dev'
    password='L@crosse753'
    user='lacrosse'
    try:
        print('loading data into database ... ')
        engine = create_engine('mysql+mysqlconnector://lacrosse:%s@182.75.105.186:33096/lacrosse-pre-dev' % quote(password))
        clubs_ranking_df.to_sql('clubs_ranking',engine,if_exists='append',index=False)
        print('success')
    except Exception as e:
        print(e)

loading data into database ... 
success
